In [9]:
from pynq import Overlay, GPIO, Register, allocate, MMIO
import os
from convert import *
import numpy as np

import struct


In [10]:
converter=Converter()

# result1 = converter.forward_conversion(input_data=-0.7)
# print('Forward converted input:',result1)

# fractional_result = converter.backward_conversion(input_data=result1)
# print('Backward converted input', fractional_result)

In [11]:
overlay = Overlay("nnv2.bit")
fccip=overlay.fcc_combined_0
convip=overlay.conv_combined_0
reluip=overlay.relu_combined_0
# overlay?
fcc1.register_map

# bck1=overlay.backward_fcc_0
# # bck1.register_map

# actv_fwd1=overlay.activation_fwd_0
# actv_bck1=overlay.activation_bckwd_0

# actv_fwd1.register_map

RegisterMap {
  CTRL = Register(AP_START=0, AP_DONE=0, AP_IDLE=1, AP_READY=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED=0),
  x = Register(x=0),
  w = Register(w=0),
  y = Register(y=0),
  b = Register(b=0),
  xdim = Register(xdim=0),
  ydim = Register(ydim=0)
}

In [12]:
class FullyConnectedLayer():
    def __init__(self,xdim,ydim,base_addr):

        self.xdim=xdim
        self.ydim=ydim

        xbuff=allocate(shape=(xdim,), dtype='uint16')
        wbuff=allocate(shape=(xdim*ydim,), dtype='uint16')
        ybuff=allocate(shape=(ydim,), dtype='uint16')
        bbuff=allocate(shape=(ydim,), dtype='uint16')
        
        dxbuff=allocate(shape=(xdim,), dtype='uint16')
        dwbuff=allocate(shape=(xdim*ydim,), dtype='uint16')
        dybuff=allocate(shape=(ydim,), dtype='uint16')
        dbbuff=allocate(shape=(ydim,), dtype='uint16')

        self.BASE_ADDRESS_X=xbuff.physical_address
        self.BASE_ADDRESS_DX=dxbuff.physical_address

        self.BASE_ADDRESS_W=wbuff.physical_address
        self.BASE_ADDRESS_DW=dwbuff.physical_address

        self.BASE_ADDRESS_Y=ybuff.physical_address
        self.BASE_ADDRESS_DY=dybuff.physical_address

        self.BASE_ADDRESS_B=bbuff.physical_address
        self.BASE_ADDRESS_DB=dbbuff.physical_address  

#         self.mmio_x= MMIO(self.BASE_ADDRESS_X,self.xdim*4)
#         self.mmio_dx=MMIO(self.BASE_ADDRESS_DX,self.xdim*4)

#         self.mmio_w= MMIO(self.BASE_ADDRESS_W,self.xdim*self.ydim*4)

#         self.mmio_y= MMIO(self.BASE_ADDRESS_Y,self.ydim*4)
#         self.mmio_dy= MMIO(self.BASE_ADDRESS_DY,self.ydim*4)

#         self.mmio_b= MMIO(self.BASE_ADDRESS_B,self.ydim*4)

#         self.config_dic={'base':base_addr, 'x':self.BASE_ADDRESS_X,'w':self.BASE_ADDRESS_W,'y':self.BASE_ADDRESS_Y,'b':self.BASE_ADDRESS_B, 'dx':self.BASE_ADDRESS_DX, 'dy':self.BASE_ADDRESS_DY,'xdim':self.xdim,'ydim':self.ydim}


    def get_config_dic(self):
        
        return self.config_dic


    def initHardware(self,fwip,lr):

        self.fwip=fwip
        self.fwip.register_map.x=self.BASE_ADDRESS_X
        self.fwip.register_map.w=self.BASE_ADDRESS_W
        self.fwip.register_map.y=self.BASE_ADDRESS_Y
        self.fwip.register_map.b=self.BASE_ADDRESS_B
        self.fwip.register_map.xdim=self.xdim
        self.fwip.register_map.ydim=self.ydim

#         self.bckip= bckip
#         self.bckip.register_map.x=self.BASE_ADDRESS_X
#         self.bckip.register_map.w=self.BASE_ADDRESS_W
#         self.bckip.register_map.b=self.BASE_ADDRESS_B
#         self.bckip.register_map.xdimension=self.xdim
#         self.bckip.register_map.ydimension=self.ydim
#         self.bckip.register_map.dx=self.BASE_ADDRESS_DX
#         self.bckip.register_map.dy=self.BASE_ADDRESS_DY
#         self.bckip.register_map.lr=lr

        
    def set_weights(self,w,b):        
        wbuff[:]=[int(converter.encode(el)) for el in w]
        bbuff[:]=[int(converter.encode(el)) for el in b]
        wbuff.flush()
        bbuff.flush()

    def reset_weights(self):
        for i in range(self.xdim*self.ydim):
            wbuff[i]=int(converter.encode(0.01))
        
        for i in range(self.ydim):
            bbuff[i]=int(converter.encode(0.01))
            
        wbuff.flush()
        bbuff.flush()
            
    def get_weights(self):
        
        w=[]
        b=[]        
        for i in range(self.xdim*self.ydim):
            w.append(converter.decode(wbuff[i]))
        
        for i in range(self.ydim):
            b.append(converter.decode(bbuff[i]))
        
        return w,b

    def set_input(self,x):
#         x=np.reshape(x,(-1,self.xdim))
        xbuff[:]=[int(converter.encode(el)) for el in x]
        xbuff.flush()
    def get_input(self):
        x=[]     
        for i in range(self.xdim):
            x.append(converter.decode(xbuff[i]))     
        return x
    def reset_input(self):
        for i in range(self.xdim):
            xbuff[i]=int(converter.encode(0.01))
            dxbuff[i]=int(converter.encode(0.0))

            
    def reset_output(self):
        for i in range(self.ydim):
            ybuff[i]=int(converter.encode(0.0))
            dybuff[i]=int(converter.encode(0.0))   
    def get_output(self):
        y=[]
        for i in range(self.ydim):
            y.append(converter.decode(ybuff[i]))
        return y

    
    
    def set_dy(self,dy):        
        for i in range(self.ydim):
            dybuff[i]=int(converter.encode(dy[i]))
    def get_dy(self):
        dy=[]
        for i in range(self.ydim):
            dy.append(converter.decode(dybuff[i]))
        return dy

    

    def fwprop(self):

        self.fwip.write(0x00, 1)
        fpga_state = self.fwip.read(0x00)

        max_try = 1000000
        while fpga_state != 6 and fpga_state != 4:
            fpga_state = self.fwip.read(0x00)
            max_try = max_try -1
            if max_try == 0:
                print("ERROR: Can't go ahead")
                self.fwip.write(0x00, 4)
                break

        self.fwip.write(0x00, 4)

    def bckprop(self):
        self.bckip.write(0x00, 1)
        fpga_state = self.bckip.read(0x00)

        max_try = 1000000
        while fpga_state != 6 and fpga_state != 4:
            fpga_state = self.bckip.read(0x00)
            max_try = max_try -1
            if max_try == 0:
                print("ERROR: Can't go ahead")
                self.bckip.write(0x00, 4)
                break

        self.bckip.write(0x00, 4)
        


In [13]:
fcc=FullyConnectedLayer(3,1,0x0000000)
fcc.initHardware(fcc1,0.1)
fcc.reset_weights()
fcc.reset_input()

In [14]:
fcc.fwprop()

In [15]:
# fcc.reset_output()
print(fcc.get_output())

[array([0.])]
